In [1]:
import sys
from pathlib import Path

import pandas as pd


sys.path.append(str(Path().resolve().parent))
from src.definitions import RAW_DATA_FOLDER

In [24]:
df = pd.read_parquet(RAW_DATA_FOLDER / "techniques-classification.parquet")

df.head(5)

,id,content,lang,manipulative,techniques,trigger_words
0,0bb0c7fa-101b-4583-a5f9-9d503339141c,Новий огляд мапи DeepState від російського вій...,uk,True,"[euphoria, loaded_language]",None
1,7159f802-6f99-4e9d-97bd-6f565a4a0fae,Недавно 95 квартал жёстко поглумился над русск...,ru,True,"[loaded_language, cherry_picking]",None
2,e6a427f1-211f-405f-bd8b-70798458d656,🤩\nТим часом йде евакуація Бєлгородського авто...,uk,True,"[loaded_language, euphoria]",None
3,1647a352-4cd3-40f6-bfa1-d87d42e34eea,В Україні найближчим часом мають намір посилит...,uk,False,None,None
4,9c01de00-841f-4b50-9407-104e9ffb03bf,"Расчёты 122-мм САУ 2С1 ""Гвоздика"" 132-й бригад...",ru,True,[loaded_language],None


In [25]:
df_uk = df[df["lang"] == "uk"]
df_ru = df[df["lang"] == "ru"]


In [ ]:
df_uk = df_uk[["id", "content"]].head(20)
df_uk

## MarianMT

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Завантажуємо модель та токенізатор
model_name = "Helsinki-NLP/opus-mt-uk-ru"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Функція для перекладу
def translate_text(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)


# Перекладаємо тексти
df_uk["translated_content"] = df_uk["content"].apply(lambda x: translate_text(x, model, tokenizer))

df_uk.to_excel("df-uk-MarianMT.xlsx", index=False)


## Google API

In [30]:
from deep_translator import GoogleTranslator

# Ініціалізація перекладача
translator = GoogleTranslator(source="uk", target="ru")

# Функція для перекладу
def translate_text(text):
    return translator.translate(text)

df_uk["translated_content"] = df_uk["content"].apply(translate_text)

df_uk.to_excel("df-uk-Google.xlsx", index=False)

## NLLB-200 from META

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Визначаємо пристрій
device = "cuda" if torch.cuda.is_available() else "cpu"

# Завантажуємо модель та токенайзер
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Функція для перекладу з укр рос
def translate_text_nllb(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["rus_Cyrl"])
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Переклад українського тексту на російську
df_uk["translated_content"] = df_uk["content"].apply(translate_text_nllb)

# Збереження в Excel
df_uk.to_excel("df-uk-NLLB-200.xlsx", index=False)



## Yandex Translate API

In [38]:
import requests

YANDEX_API_KEY = "y0__xDlzqj6BxjB3RMgkeXJqRJHVWVXWpDoscZgDfTZirB4c33k4Q"

def translate_text_yandex(text):
    url = "https://translate.api.cloud.yandex.net/translate/v2/translate"
    headers = {"Authorization": f"Api-Key {YANDEX_API_KEY}"}
    data = {
        "folder_id": "your_folder_id",
        "texts": [text],
        "sourceLanguageCode": "uk",
        "targetLanguageCode": "ru"
    }
    response = requests.post(url, json=data, headers=headers)
    return response.json()["translations"][0]["text"]

df_uk["translated_content"] = df_uk["content"].apply(translate_text_yandex)


KeyError: 'translations'

In [39]:
import requests
import json

# Ваш OAuth токен
oauth_token = "y0__xDlzqj6BxjB3RMgkeXJqRJHVWVXWpDoscZgDfTZirB4c33k4Q"

# URL для запиту
url = "https://iam.api.cloud.yandex.net/iam/v1/tokens"

# Дані для запиту
data = {
    "yandexPassportOauthToken": oauth_token
}

# Виконання POST-запиту
response = requests.post(url, json=data)

# Перевірка статусу відповіді
if response.status_code == 200:
    print("Токен правильний!")
    # Можна отримати доступ до даних відповіді
    token_info = response.json()
    print("Інформація про токен:", token_info)
else:
    print("Токен неправильний або сталася помилка.")
    print("Статус код:", response.status_code)
    print("Відповідь:", response.text)


Токен неправильний або сталася помилка.
Статус код: 401
Відповідь: {
 "code": 16,
 "message": "User does not registered in Yandex.cloud",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "12fa304c-40c1-459e-9272-a8fd314888a2"
  }
 ]
}

